In [1]:
!nvidia-smi

Sat Oct 19 16:31:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              10W /  70W |      5MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
pip install pandas

In [ ]:
pip install mpi4py

In [ ]:
pip install transformers seqeval[gpu]

In [2]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


https://data.world/collery/post-office-data/workspace/file?filename=USPO+Change+of+Address+data+Y2022.csv
https://data.world/len/us-first-names-database

https://www.kaggle.com/datasets/ahmedshahriarsakib/list-of-real-usa-addresses
https://www.kaggle.com/datasets/datagov/usa-names

https://data.world/adamhelsinger/directory-of-representatives
https://data.world/state-of-connecticut/ikyf-f2iu

In [3]:
import pandas as pd


In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [5]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [6]:
# Read the CSV file
surnames = pd.read_csv("data/Common_Surnames_Census_2000.csv")

# View the first 5 rows
surnames.head()

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1.0,2376206.0,880.85,880.85,73.35,22.22,0.40,0.85,1.63,1.56
1,JOHNSON,2.0,1857160.0,688.44,1569.30,61.55,33.80,0.42,0.91,1.82,1.50
2,WILLIAMS,3.0,1534042.0,568.66,2137.96,48.52,46.72,0.37,0.78,2.01,1.60
3,BROWN,4.0,1380145.0,511.62,2649.58,60.71,34.54,0.41,0.83,1.86,1.64
4,JONES,5.0,1362755.0,505.17,3154.75,57.69,37.73,0.35,0.94,1.85,1.44


In [7]:
surnames = surnames.drop(['rank', 'count', 'prop100k', 'cum_prop100k', 'pctwhite', 'pctblack', 'pctapi', 'pctaian', 'pct2prace', 'pcthispanic'], axis=1)

In [8]:
surnames.shape

(85502, 1)

In [9]:
surnames.head()

,name
0,SMITH
1,JOHNSON
2,WILLIAMS
3,BROWN
4,JONES


In [10]:
surnames = surnames.drop_duplicates(subset=['name'])

In [11]:
surnames = surnames.rename(columns={"name": "lastname"})

In [17]:
# Read the CSV file
firstnameandgender = pd.read_csv("data/SSA_Names_DB.csv")

# View the first 5 rows
firstnameandgender.head()

,Name,Gender,Frequency,Include?
0,Emma,F,20355,Yes
1,Olivia,F,19553,Yes
2,Noah,M,19511,Yes
3,Liam,M,18281,Yes
4,Sophia,F,17327,Yes


In [18]:
firstnameandgender = firstnameandgender.drop(['Frequency', 'Include?',], axis=1)

In [19]:
firstnameandgender.shape

(32952, 2)

In [20]:
firstnameandgender = firstnameandgender.drop_duplicates(subset=['Name'])

In [21]:
firstnameandgender.shape

(30460, 2)

In [22]:
firstnameandgender = firstnameandgender.rename(columns={"Name": "firstname", "Gender": "gender"})

In [23]:
firstnameandgender.shape

(30460, 2)

In [24]:
firstnameandgender.head()

,firstname,gender
0,Emma,F
1,Olivia,F
2,Noah,M
3,Liam,M
4,Sophia,F


In [25]:
# Read the CSV file
address = pd.read_csv("data/USPO Change of Address data Y2022.csv")

# View the first 5 rows
address.head()

,YYYYMM,ZIPCODE,CITY,STATE,TOTAL FROM ZIP,TOTAL BUSINESS,TOTAL FAMILY,TOTAL INDIVIDUAL,TOTAL PERM,TOTAL TEMP,TOTAL TO ZIP,TOTAL BUSINESS.1,TOTAL FAMILY.1,TOTAL INDIVIDUAL.1,TOTAL PERM.1,TOTAL TEMP.1
0,202201,"=""00601""",ADJUNTAS,PR,0,0,0,0,0,0,13,0,0,11,12,0
1,202201,"=""00602""",AGUADA,PR,18,0,0,14,17,0,43,0,0,39,30,13
2,202201,"=""00603""",AGUADILLA,PR,31,0,0,24,24,0,65,0,11,51,40,25
3,202201,"=""00605""",AGUADILLA,PR,13,0,0,12,12,0,11,0,0,0,11,0
4,202201,"=""00610""",ANASCO,PR,11,0,0,0,0,0,17,0,0,0,0,0


In [26]:
address = address.drop(['YYYYMM', 'TOTAL FROM ZIP',"TOTAL BUSINESS", "TOTAL FAMILY", "TOTAL INDIVIDUAL", "TOTAL PERM", "TOTAL TEMP"], axis=1)

In [27]:
address = address.drop(['TOTAL TO ZIP', 'TOTAL BUSINESS.1',"TOTAL FAMILY.1", "TOTAL INDIVIDUAL.1", "TOTAL PERM.1", "TOTAL TEMP.1"], axis=1)

In [28]:
address.head()

,ZIPCODE,CITY,STATE
0,"=""00601""",ADJUNTAS,PR
1,"=""00602""",AGUADA,PR
2,"=""00603""",AGUADILLA,PR
3,"=""00605""",AGUADILLA,PR
4,"=""00610""",ANASCO,PR


In [29]:
address['ZIPCODE'] = address['ZIPCODE'].apply(lambda x: x.replace('=', '').replace('"', ''))


In [30]:
address.head()

,ZIPCODE,CITY,STATE
0,00601,ADJUNTAS,PR
1,00602,AGUADA,PR
2,00603,AGUADILLA,PR
3,00605,AGUADILLA,PR
4,00610,ANASCO,PR


In [31]:
address = address.rename(columns={"ZIPCODE": "zipcode", "CITY": "city", "STATE": "state"})

In [32]:
address.shape

(278400, 3)

In [33]:
lst=address.city.unique() 

In [34]:
len(lst)

14806

In [35]:
# shuffle the data
address = address.sample(frac = 1)
firstnameandgender = firstnameandgender.sample(frac = 1)
surnames = surnames.sample(frac = 1)

In [36]:
address.reset_index(inplace = True,  drop=True)
firstnameandgender.reset_index(inplace = True,  drop=True)
surnames.reset_index(inplace = True,  drop=True)


In [37]:
surnames=surnames.iloc[:30460]

In [38]:
firstnameandgender=firstnameandgender.iloc[:30460]

In [39]:
address=address.iloc[:30460]

In [40]:
address.shape

(30460, 3)

In [41]:
horizontal_concat = pd.concat([firstnameandgender, surnames, address], axis=1)


In [42]:
horizontal_concat.head()

,firstname,gender,lastname,zipcode,city,state
0,Lanya,F,HAMMA,35640,HARTSELLE,AL
1,Ellanora,F,PATTENGALE,07417,FRANKLIN LAKES,NJ
2,Riya,F,TOBIASZ,33621,TAMPA,FL
3,Briza,F,LAGACE,30070,PORTERDALE,GA
4,Prosperity,F,PAPENDICK,12053,DELANSON,NY


In [43]:
horizontal_concat.reset_index(inplace=True, drop=True) 


In [44]:
horizontal_concat.shape

(30460, 6)

In [45]:
horizontal_concat['firstname'] = horizontal_concat['firstname'].str.lower()

In [46]:
horizontal_concat['gender'] = horizontal_concat['gender'].str.lower()

In [47]:
horizontal_concat['lastname'] = horizontal_concat['lastname'].str.lower()

In [48]:
horizontal_concat['city'] = horizontal_concat['city'].str.lower()

In [49]:
horizontal_concat['state'] = horizontal_concat['state'].str.lower()

In [50]:
horizontal_concat

,firstname,gender,lastname,zipcode,city,state
0,lanya,f,hamma,35640,hartselle,al
1,ellanora,f,pattengale,07417,franklin lakes,nj
2,riya,f,tobiasz,33621,tampa,fl
3,briza,f,lagace,30070,porterdale,ga
4,prosperity,f,papendick,12053,delanson,ny
...,...,...,...,...,...,...
30455,arizona,f,miu,25921,sophia,wv
30456,mayci,f,fausey,57754,lead,sd
30457,yuto,m,zemke,12816,cambridge,ny
30458,jentzen,m,dorosh,80932,colorado springs,co


In [51]:
#Feature Engineering
# Melt the DataFrame to convert columns to rows
melted_data = pd.melt(horizontal_concat, var_name='Entity', value_name='Value')
 
# 'Entity' column now contains the labels for the entities (former column names)
# 'Value' column contains the corresponding values
 
print(melted_data)

           Entity       Value
0       firstname       lanya
1       firstname    ellanora
2       firstname        riya
3       firstname       briza
4       firstname  prosperity
...           ...         ...
182755      state          wv
182756      state          sd
182757      state          ny
182758      state          co
182759      state          fl

[182760 rows x 2 columns]


In [52]:
melted_data['Entity'].unique()


array(['firstname', 'gender', 'lastname', 'zipcode', 'city', 'state'],
      dtype=object)

In [53]:
# Define the labels for each column
labels = ['firstname', 'gender', 'lastname', 'zipcode', 'city', 'state']


In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming your melted data is stored in a DataFrame called 'melted_data'
# and the labels are in a column named 'Value'

entity_lists = melted_data['Entity'].unique()
X_test_final = pd.DataFrame()
X_train_final = pd.DataFrame()
X_val_final = pd.DataFrame()
y_train_final = pd.Series()
y_test_final = pd.Series()
y_val_final = pd.Series()

for entity in entity_lists:
    # Filter data for the current entity
    entity_data = melted_data[melted_data['Entity'] == entity]
    
    # Split into features and labels
   # X = entity_data.drop('Value', axis=1)  # Assuming 'Value' is the label column
    X = entity_data['Value']
    y = entity_data['Entity']
    
    # Split into train, test, and validation sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    # Append the splits to the final DataFrames and Series
    X_train_final = pd.concat([X_train_final, X_train], ignore_index=True)
    X_test_final = pd.concat([X_test_final, X_test], ignore_index=True)
    X_val_final = pd.concat([X_val_final, X_val], ignore_index=True)
    
    y_train_final = pd.concat([y_train_final, y_train], ignore_index=True)
    y_test_final = pd.concat([y_test_final, y_test], ignore_index=True)
    y_val_final = pd.concat([y_val_final, y_val], ignore_index=True)

    # Display the sizes of the splits for the current entity
    print(f"Entity: {entity}")
    print("Training set size:", len(X_train))
    print("Testing set size:", len(X_test))
    print("Validation set size:", len(X_val))
    print("===================")




Entity: firstname
Training set size: 21322
Testing set size: 4569
Validation set size: 4569
Entity: gender
Training set size: 21322
Testing set size: 4569
Validation set size: 4569
Entity: lastname
Training set size: 21322
Testing set size: 4569
Validation set size: 4569
Entity: zipcode
Training set size: 21322
Testing set size: 4569
Validation set size: 4569
Entity: city
Training set size: 21322
Testing set size: 4569
Validation set size: 4569
Entity: state
Training set size: 21322
Testing set size: 4569
Validation set size: 4569


In [55]:
print("X_test_final", X_test_final)
print("X_train_final", X_train_final)
print("X_val_final", X_val_final)
print("y_train_final", y_train_final)
print("y_test_final", y_test_final)
print("y_val_final", y_val_final)

X_test_final               0
0       quintez
1           jed
2         noora
3         adara
4      shreshta
...         ...
27409        ky
27410        mi
27411        mi
27412        tx
27413        fl

[27414 rows x 1 columns]
X_train_final              0
0       sharvi
1       kaicen
2       taryah
3        mayte
4        ivaan
...        ...
127927      nm
127928      pa
127929      wi
127930      tx
127931      tn

[127932 rows x 1 columns]
X_val_final              0
0         adda
1       fallou
2        blimy
3      bostynn
4       brysyn
...        ...
27409       va
27410       oh
27411       wi
27412       va
27413       ks

[27414 rows x 1 columns]
y_train_final 0         firstname
1         firstname
2         firstname
3         firstname
4         firstname
            ...    
127927        state
127928        state
127929        state
127930        state
127931        state
Length: 127932, dtype: object
y_test_final 0        firstname
1        firstname
2        firstn

In [56]:
print("X_test_final", len(X_test_final))
print("X_train_final", len(X_train_final))
print("X_val_final", len(X_val_final))
print("y_train_final", len(y_train_final))
print("y_test_final", len(y_test_final))
print("y_val_final", len(y_val_final))

X_test_final 27414
X_train_final 127932
X_val_final 27414
y_train_final 127932
y_test_final 27414
y_val_final 27414


In [57]:
entity_data = melted_data[melted_data['Entity'] == 'firstname']

In [58]:
entity_data.head()

,Entity,Value
0,firstname,lanya
1,firstname,ellanora
2,firstname,riya
3,firstname,briza
4,firstname,prosperity


In [59]:
X_train_final.columns

Index([0], dtype='int64')

In [60]:
!pip install transformers seqeval[gpu]

In [61]:
pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.


In [62]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [63]:
import transformers
print(transformers.__version__)


4.45.2


In [64]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [65]:
frequencies=melted_data['Entity'].unique()


In [66]:
frequencies

array(['firstname', 'gender', 'lastname', 'zipcode', 'city', 'state'],
      dtype=object)

In [67]:
labels_to_ids = {k: v for v, k in enumerate(frequencies)}
ids_to_labels = {v: k for v, k in enumerate(frequencies)}
labels_to_ids

{'firstname': 0,
 'gender': 1,
 'lastname': 2,
 'zipcode': 3,
 'city': 4,
 'state': 5}

In [68]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.Value[index].strip().split()  
        word_labels = [self.data.Entity[index]]

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             is_split_into_words=True, 
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [69]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 512
VALID_BATCH_SIZE = 128
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [70]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [71]:
melted_data

,Entity,Value
0,firstname,lanya
1,firstname,ellanora
2,firstname,riya
3,firstname,briza
4,firstname,prosperity
...,...,...
182755,state,wv
182756,state,sd
182757,state,ny
182758,state,co


In [72]:
train_size = 0.8
train_dataset = melted_data.sample(frac=train_size,random_state=200)
test_dataset = melted_data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(melted_data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (182760, 2)
TRAIN Dataset: (146208, 2)
TEST Dataset: (36552, 2)


//Dump : Pickle dump file
// learning rate training accuracy  + validation accuracy : line plot

In [73]:

training_set[0]

{'input_ids': tensor([  101,  2128, 16368,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [74]:
labels_to_ids

{'firstname': 0,
 'gender': 1,
 'lastname': 2,
 'zipcode': 3,
 'city': 4,
 'state': 5}

In [75]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["input_ids"]), training_set[0]["labels"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       -100
re          2
##hor       -100
[SEP]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100


In [76]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [77]:
# configuration = BertConfig(vocab_size=30_522)
# model = BertForTokenClassification(config=configuration).from_pretrained('bert-base-cased', num_labels=len(labels_to_ids))

model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids), id2label=ids_to_labels,label2id=labels_to_ids)
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [78]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(2.0705, device='cuda:0', grad_fn=<NllLossBackward0>)

In [79]:
len(labels)

1

In [80]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 6])

In [81]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [82]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    print("training_loader*****")
    print(training_loader)
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    print(" model.train()*****")
    for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)
        print(" batch['labels']*****")
        print(labels)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [83]:
EPOCHS

1

In [84]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
training_loader*****
 model.train()*****


IndexError: list index out of range

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=targets)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)